In [6]:
# Copyright 2013, Michael H. Goldwasser
#
# Developed for use with the book:
#
#    Data Structures and Algorithms in Python
#    Michael T. Goodrich, Roberto Tamassia, and Michael H. Goldwasser
#    John Wiley & Sons, 2013
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>.

#from ..ch07.linked_queue import LinkedQueue
import collections

class Tree:
  """Abstract base class representing a tree structure."""

  #------------------------------- nested Position class -------------------------------
  class Position:
    """An abstraction representing the location of a single element within a tree.

    Note that two position instaces may represent the same inherent location in a tree.
    Therefore, users should always rely on syntax 'p == q' rather than 'p is q' when testing
    equivalence of positions.
    """

    def element(self):
      """Return the element stored at this Position."""
      raise NotImplementedError('must be implemented by subclass')
      
    def __eq__(self, other):
      """Return True if other Position represents the same location."""
      raise NotImplementedError('must be implemented by subclass')

    def __ne__(self, other):
      """Return True if other does not represent the same location."""
      return not (self == other)            # opposite of __eq__

  # ---------- abstract methods that concrete subclass must support ----------
  def root(self):
    """Return Position representing the tree's root (or None if empty)."""
    raise NotImplementedError('must be implemented by subclass')

  def parent(self, p):
    """Return Position representing p's parent (or None if p is root)."""
    raise NotImplementedError('must be implemented by subclass')

  def num_children(self, p):
    """Return the number of children that Position p has."""
    raise NotImplementedError('must be implemented by subclass')

  def children(self, p):
    """Generate an iteration of Positions representing p's children."""
    raise NotImplementedError('must be implemented by subclass')

  def __len__(self):
    """Return the total number of elements in the tree."""
    raise NotImplementedError('must be implemented by subclass')

  # ---------- concrete methods implemented in this class ----------
  def is_root(self, p):
    """Return True if Position p represents the root of the tree."""
    return self.root() == p

  def is_leaf(self, p):
    """Return True if Position p does not have any children."""
    return self.num_children(p) == 0

  def is_empty(self):
    """Return True if the tree is empty."""
    return len(self) == 0

  def depth(self, p):
    """Return the number of levels separating Position p from the root."""
    if self.is_root(p):
      return 0
    else:
      return 1 + self.depth(self.parent(p))

  def _height1(self):                 # works, but O(n^2) worst-case time
    """Return the height of the tree."""
    return max(self.depth(p) for p in self.positions() if self.is_leaf(p))

  def _height2(self, p):                  # time is linear in size of subtree
    """Return the height of the subtree rooted at Position p."""
    if self.is_leaf(p):
      return 0
    else:
      return 1 + max(self._height2(c) for c in self.children(p))

  def height(self, p=None):
    """Return the height of the subtree rooted at Position p.

    If p is None, return the height of the entire tree.
    """
    if p is None:
      p = self.root()
    return self._height2(p)        # start _height2 recursion

  def __iter__(self):
    """Generate an iteration of the tree's elements."""
    for p in self.positions():                        # use same order as positions()
      yield p.element()                               # but yield each element

  def positions(self):
    """Generate an iteration of the tree's positions."""
    return self.preorder()                            # return entire preorder iteration

  def preorder(self):
    """Generate a preorder iteration of positions in the tree."""
    if not self.is_empty():
      for p in self._subtree_preorder(self.root()):  # start recursion
        yield p

  def _subtree_preorder(self, p):
    """Generate a preorder iteration of positions in subtree rooted at p."""
    yield p                                           # visit p before its subtrees
    for c in self.children(p):                        # for each child c
      for other in self._subtree_preorder(c):         # do preorder of c's subtree
        yield other                                   # yielding each to our caller

  def postorder(self):
    """Generate a postorder iteration of positions in the tree."""
    if not self.is_empty():
      for p in self._subtree_postorder(self.root()):  # start recursion
        yield p

  def _subtree_postorder(self, p):
    """Generate a postorder iteration of positions in subtree rooted at p."""
    for c in self.children(p):                        # for each child c
      for other in self._subtree_postorder(c):        # do postorder of c's subtree
        yield other                                   # yielding each to our caller
    yield p                                           # visit p after its subtrees

  def breadthfirst(self):
    """Generate a breadth-first iteration of the positions of the tree."""
    if not self.is_empty():
      fringe = LinkedQueue()             # known positions not yet yielded
      fringe.enqueue(self.root())        # starting with the root
      while not fringe.is_empty():
        p = fringe.dequeue()             # remove from front of the queue
        yield p                          # report this position
        for c in self.children(p):
          fringe.enqueue(c)              # add children to back of queue


In [7]:
# Copyright 2013, Michael H. Goldwasser
#
# Developed for use with the book:
#
#    Data Structures and Algorithms in Python
#    Michael T. Goodrich, Roberto Tamassia, and Michael H. Goldwasser
#    John Wiley & Sons, 2013
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>.

def toc_plain(T):
  for p in T.preorder():
    print(p.element())

def toc_indent_bad(T):
  for p in T.preorder():
    print(2*T.depth(p)*' ' + str(p.element()))  # beware of inefficiency

def preorder_indent(T, p, d):
  """Print preorder representation of subtree of T rooted at p at depth d."""
  print(2*d*' ' + str(p.element()))           # use depth for indentation
  for c in T.children(p):
    preorder_indent(T, c, d+1)                # child depth is d+1

def preorder_label(T, p, d, path):
  """Print labeled representation of subtree of T rooted at p at depth d."""
  label = '.'.join(str(j+1) for j in path)    # displayed labels are one-indexed
  print(2*d*' ' + label, p.element())
  path.append(0)                              # path entries are zero-indexed
  for c in T.children(p):
    preorder_label(T, c, d+1, path)           # child depth is d+1
    path[-1] += 1
  path.pop()

def parenthesize(T, p):
  """Print parenthesized representation of subtree of T rooted at p."""
  print(p.element(), end='')                  # use of end avoids trailing newline
  if not T.is_leaf(p):
    first_time = True
    for c in T.children(p):
      sep = ' (' if first_time else ', '      # determine proper separator
      print(sep, end='')        
      first_time = False                      # any future passes will not be the first
      parenthesize(T, c)                      # recur on child
    print(')', end='')                        # include closing parenthesis

def disk_space(T, p):
  """Return total disk space for subtree of T rooted at p."""
  subtotal = p.element().space()              # space used at position p
  for c in T.children(p):
    subtotal += disk_space(T, c)              # add child's space to subtotal
  return subtotal


In [8]:
# Copyright 2013, Michael H. Goldwasser
#
# Developed for use with the book:
#
#    Data Structures and Algorithms in Python
#    Michael T. Goodrich, Roberto Tamassia, and Michael H. Goldwasser
#    John Wiley & Sons, 2013
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>.

#from .tree import Tree

class BinaryTree(Tree):
  """Abstract base class representing a binary tree structure."""

  # --------------------- additional abstract methods ---------------------
  def left(self, p):
    """Return a Position representing p's left child.

    Return None if p does not have a left child.
    """
    raise NotImplementedError('must be implemented by subclass')

  def right(self, p):
    """Return a Position representing p's right child.

    Return None if p does not have a right child.
    """
    raise NotImplementedError('must be implemented by subclass')

  # ---------- concrete methods implemented in this class ----------
  def sibling(self, p):
    """Return a Position representing p's sibling (or None if no sibling)."""
    parent = self.parent(p)
    if parent is None:                    # p must be the root
      return None                         # root has no sibling
    else:
      if p == self.left(parent):
        return self.right(parent)         # possibly None
      else:
        return self.left(parent)          # possibly None

  def children(self, p):
    """Generate an iteration of Positions representing p's children."""
    if self.left(p) is not None:
      yield self.left(p)
    if self.right(p) is not None:
      yield self.right(p)

  def inorder(self):
    """Generate an inorder iteration of positions in the tree."""
    if not self.is_empty():
      for p in self._subtree_inorder(self.root()):
        yield p

  def _subtree_inorder(self, p):
    """Generate an inorder iteration of positions in subtree rooted at p."""
    if self.left(p) is not None:          # if left child exists, traverse its subtree
      for other in self._subtree_inorder(self.left(p)):
        yield other
    yield p                               # visit p between its subtrees
    if self.right(p) is not None:         # if right child exists, traverse its subtree
      for other in self._subtree_inorder(self.right(p)):
        yield other

  # override inherited version to make inorder the default
  def positions(self):
    """Generate an iteration of the tree's positions."""
    return self.inorder()                 # make inorder the default


In [9]:
# Copyright 2013, Michael H. Goldwasser
#
# Developed for use with the book:
#
#    Data Structures and Algorithms in Python
#    Michael T. Goodrich, Roberto Tamassia, and Michael H. Goldwasser
#    John Wiley & Sons, 2013
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>.

class EulerTour:
  """Abstract base class for performing Euler tour of a tree.

  _hook_previsit and _hook_postvisit may be overridden by subclasses.
  """
  def __init__(self, tree):
    """Prepare an Euler tour template for given tree."""
    self._tree = tree

  def tree(self):
    """Return reference to the tree being traversed."""
    return self._tree

  def execute(self):
    """Perform the tour and return any result from post visit of root."""
    if len(self._tree) > 0:
      return self._tour(self._tree.root(), 0, [])    # start the recursion

  def _tour(self, p, d, path):
    """Perform tour of subtree rooted at Position p.

    p        Position of current node being visited
    d        depth of p in the tree
    path     list of indices of children on path from root to p
    """
    self._hook_previsit(p, d, path)                       # "pre visit" p
    results = []
    path.append(0)          # add new index to end of path before recursion
    for c in self._tree.children(p):
      results.append(self._tour(c, d+1, path))  # recur on child's subtree
      path[-1] += 1         # increment index
    path.pop()              # remove extraneous index from end of path
    answer = self._hook_postvisit(p, d, path, results)    # "post visit" p
    return answer

  def _hook_previsit(self, p, d, path):
    """Visit Position p, before the tour of its children.

    p        Position of current position being visited
    d        depth of p in the tree
    path     list of indices of children on path from root to p
    """
    pass

  def _hook_postvisit(self, p, d, path, results):
    """Visit Position p, after the tour of its children.

    p        Position of current position being visited
    d        depth of p in the tree
    path     list of indices of children on path from root to p
    results  is a list of values returned by _hook_postvisit(c)
            for each child c of p.
    """
    pass

class PreorderPrintIndentedTour(EulerTour):
  def _hook_previsit(self, p, d, path):
   print(2*d*' ' + str(p.element()))

class PreorderPrintIndentedLabeledTour(EulerTour):
  def _hook_previsit(self, p, d, path):
    label = '.'.join(str(j+1) for j in path)    # labels are one-indexed
    print(2*d*' ' + label, p.element())

class ParenthesizeTour(EulerTour):
  def _hook_previsit(self, p, d, path):
    if path and path[-1] > 0:                  # p follows a sibling
      print(', ', end='')                      # so preface with comma
    print(p.element(), end='')                 # then print element
    if not self.tree().is_leaf(p):             # if p has children
      print(' (', end='')                      # print opening parenthesis

  def _hook_postvisit(self, p, d, path, results):
    if not self.tree().is_leaf(p):             # if p has children
      print(')', end='')                       # print closing parenthesis

class DiskSpaceTour(EulerTour):
  def _hook_postvisit(self, p, d, path, results):
    # we simply add space associated with p to that of its subtrees
    return p.element().space() + sum(results)   

class BinaryEulerTour(EulerTour):
  """Abstract base class for performing Euler tour of a binary tree.

  This version includes an additional _hook_invisit that is called after the tour
  of the left subtree (if any), yet before the tour of the right subtree (if any).

  Note: Right child is always assigned index 1 in path, even if no left sibling.
  """
  def _tour(self, p, d, path):
    results = [None, None]          # will update with results of recursions
    self._hook_previsit(p, d, path)                  # "pre visit" for p
    if self._tree.left(p) is not None:               # consider left child
      path.append(0)
      results[0] = self._tour(self._tree.left(p), d+1, path)
      path.pop()
    self._hook_invisit(p, d, path)                   # "in visit" for p
    if self._tree.right(p) is not None:              # consider right child
      path.append(1)
      results[1] = self._tour(self._tree.right(p), d+1, path)
      path.pop()
    answer = self._hook_postvisit(p, d, path, results)    # "post visit" p
    return answer

  def _hook_invisit(self, p, d, path):
    """Visit Position p, between tour of its left and right subtrees."""
    pass

class BinaryLayout(BinaryEulerTour):
  """Class for computing (x,y) coordinates for each node of a binary tree."""
  def __init__(self, tree):
    super().__init__(tree)           # must call the parent constructor
    self._count = 0                  # initialize count of processed nodes

  def _hook_invisit(self, p, d, path):
    p.element().setX(self._count)    # x-coordinate serialized by count
    p.element().setY(d)              # y-coordinate is depth
    self._count += 1                 # advance count of processed nodes
